## 摘要 Abstract

- 我们训练了一个大型、深度卷积神经网络，将 ImageNet LSVRC-2010 竞赛中的 120 万张高分辨率图像分类到 1000 个不同的类别中。

- 在测试数据上，我们实现了 37.5%的 top-1 错误率和 17.0%的 top-5 错误率，这比之前最先进的技术有显著提升。
- 这个拥有 6000 万个参数和 65 万个神经元的神经网络由五个卷积层组成，其中一些层后面跟着最大池化层，以及三个全连接层，最终是一个 1000 路的 softmax。
- 为了加快训练速度，我们使用了非饱和神经元和非常高效的 GPU 卷积运算实现。为了减少全连接层的过拟合，我们采用了一种最近开发出来的正则化方法，称为“dropout”，证明非常有效。
- 我们还将这个模型的变体参加了 ILSVRC-2012 竞赛，并取得了 15.3%的 top-5 测试错误率，相比之下，第二名的错误率为 26.2%。

## 引言 Introduction

## 数据集 Dataset

- ImageNet 是一个包含超过 1500 万张标记的高分辨率图像的数据集，这些图像属于大约 22,000 个类别。
<!-- - 这些图像通过网络收集，并由使用亚马逊的 Mechanical Turk 众包工具的人类标注员进行标记。从 2010 年开始，作为 Pascal 视觉对象挑战的一部分，每年都会举办一个名为 ImageNet 大规模视觉识别挑战（ILSVRC）的年度竞赛。ILSVRC 使用 ImageNet 的一个子集，每个类别中约有 1000 张图像。总共有大约 120 万张训练图像，50000 张验证图像和 150000 张测试图像。 -->

<!-- ILSVRC-2010 是唯一一个测试集标签可用的 ILSVRC 版本，因此这是我们进行大部分实验的版本。由于我们也参加了 ILSVRC-2012 比赛，在第六节中，我们也报告了该数据集版本的结果，其测试集标签不可用。在 ImageNet 上，通常报告两种错误率：top-1 和 top-5，其中 top-5 错误率是指模型认为最可能的五个标签中不包括正确标签的测试图像的比例。 -->

- ImageNet 包含不同分辨率的图像，而我们的系统需要恒定的输入维度。因此，我们将图像下采样到固定的 256×256 分辨率。
  - 下采样(Resizing & Cropping): 缩放使短边为 256，然后中心裁剪为 256×256
<!-- 对于一个矩形图像，我们首先将其缩放，使较短边长度为 256，然后从得到的图像中裁剪出中心的 256×256 块。除了从每个像素中减去训练集的平均活动值外，我们没有以任何其他方式预处理图像。因此，我们在（居中的）原始 RGB 像素值上训练我们的网络。 -->

## 架构 Architecture

### ReLU 非线性

- 常规的方式是用:
  - $f(x) = tanh(x)$
  - $f(x) = \frac{1}{1+e^{-x}}$
- 我们使用:
  - $f(x) = max(0, x)$ (RELU)
- ReLU 比饱和非线性函数收敛更快。

### 局部响应归一化 Local Response Normalization

$$\begin{aligned}
b_{x,y}^i = a_{x, y}^i / \left( k + \alpha \sum_{j=\max(0, i - n/2)}^{\min(N-1, i + n/2)} (a_{x,y}^j)^2 \right)^\beta
\end{aligned}$$

<!-- 相邻n个通道在x,y位置的激活值 平方和再乘以alpha + k, 再beta幂次，用原来的激活值/结果 -->
- $a_{x,y}^i$ 第i个通道上位置(x,y)的(RELU)激活值
- $j$表示以$i$为中心的邻域内的通道索引
- 相邻通道的$(x,y)$位置的激活值会抑制$a_{x,y}^i$的响应，这种机制类似于生物神经系统中的侧抑制现象，有助于增强模型的泛化能力。

### 整体框架

- [图 1](#alexnet_arch) 清楚展示了两个GPU的职责。
- 八个带权重的层: 五个卷积层和三个全连接层。

<a id="alexnet_arch"></a>
<div style="background-color: white; padding: 10px; border-radius: 5px; width: 80%; margin: auto;">
    <image src="./assets/AlexNet_arch.png" alt="AlexNet Architecture" style="max-width: 100%; height: auto;">
    <p style="text-align: center; font-style: italic;">图 1:
    AlexNet 架构示意图</p>
</div>

| 层类型               | 层名称       | 卷积核/池化核尺寸 | 步长 | 填充 | 输入维度          | 输出维度          | 其他参数（数量/激活）       |
|----------------------|--------------|-------------------|------|------|-------------------|-------------------|----------------------------|
| 输入层               | Input        | -                 | -    | -    | 227×227×3         | 227×227×3         | RGB 彩色图像                |
| 卷积层               | Conv1        | 11×11             | 4    | 2    | 227×227×3         | 55×55×96          | 96 (48×2) 个卷积核，ReLU 激活；双 GPU 拆分（conv1a/conv1b 各 48 核）       |
| 响应归一化层       | LRN1        | 5x5(归一化窗口)                 | -    | -    | 55×55×96          | 55×55×96          | 局部响应归一化（LRN）；$\alpha=0.0001，\beta=0.75，k=2$（AlexNet 标准参数）              |
| 池化层（最大池化）| Max Pool1    | 3×3               | 2    | 0    | 55×55×96          | 27×27×96          | 重叠池化                    |
| 卷积层               | Conv2        | 5×5               | 1    | 2    | 27×27×96          | 27×27×256         | 256 (128×2) 个卷积核，ReLU 激活；双 GPU 拆分（conv2a/conv2b 各 128 核） |
| 响应归一化层       | LRN2        | 5x5(归一化窗口)                 | -    | -    | 27×27×256         | 27×27×256         | 局部响应归一化（LRN）；$\alpha=0.0001，\beta=0.75，k=2$（AlexNet 标准参数）              |
| 池化层（最大池化）| Max Pool2    | 3×3               | 2    | 0    | 27×27×256         | 13×13×256         | 重叠池化                    |
| 卷积层               | Conv3        | 3×3               | 1    | 1    | 13×13×256         | 13×13×384         | 384 (192×2) 个卷积核，ReLU 激活；双 GPU 拆分（conv3a/conv3b 各 192 核）      |
| 卷积层               | Conv4        | 3×3               | 1    | 1    | 13×13×384         | 13×13×384         | 384 (192×2) 个卷积核，ReLU 激活；双 GPU 拆分（conv4a/conv4b 各 192 核） |
| 卷积层               | Conv5        | 3×3               | 1    | 1    | 13×13×384         | 13×13×256         | 256 (128×2) 个卷积核，ReLU 激活；双 GPU 拆分（conv5a/conv5b 各 128 核） |
| 池化层（最大池化）| Max Pool3    | 3×3               | 2    | 0    | 13×13×256         | 6×6×256           | 重叠池化                    |
| 全连接层             | FC6          | -                 | -    | -    | 6×6×256=9216      | 4096              | ReLU 激活 + Dropout(0.5)，双 GPU 拆分（fc2048a/fc2048b 各 2048 维）    |
| 全连接层             | FC7          | -                 | -    | -    | 4096              | 4096              | ReLU 激活 + Dropout(0.5)，双 GPU 拆分（fc2048ba/fc2048bb 各 2048 维）    |
| 全连接层（输出层）| FC8          | -                 | -    | -    | 4096              | 1000              | Softmax 激活（ImageNet 1000 分类）|